In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Competition link = Alexander Novy, CH1Mercedes, Christian Drescher, Christian Pfaundler, KOESIM, Will Cukierski. (2017). Mercedes-Benz Greener Manufacturing. Kaggle. https://kaggle.com/competitions/mercedes-benz-greener-manufacturing

In [ ]:
pd.options.display.max_columns=400

# Importing necessary datasets and packages 

In [ ]:
train=pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/train.csv.zip")
train

In [ ]:
#Check the distribution of the outcome variable
import seaborn as sns
sns.displot(train["y"])

In [ ]:
#See if log transformation of the outcome variable can change the distribution close to normal distribution
sns.displot(np.log(train["y"]))

In [ ]:
test=pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/test.csv.zip")
test

# Data Pre-processing

In [ ]:
#Check if there are any null values => None 
train[train.isnull().any(axis=1)]



In [ ]:
test[test.isnull().any(axis=1)]

In [ ]:
#Merge Train and Test datasets to ease the process of the data pre-processing/remove all the data
all_data=pd.concat([train,test])
all_data2=all_data.drop(columns=["y","ID"])
all_data2= all_data2.loc[:, (train != 0).any()]
all_data2


In [ ]:
#Automatic encoding process for columns that do not have binary values but alphabet values 
from sklearn.preprocessing import LabelEncoder 
le=LabelEncoder()

#Encode all the cateogrical variables
cat_features=all_data2.columns[all_data2.dtypes==object]
cat_features

In [ ]:
#Apply encoding process for all columns with categorical variables
for i in cat_features:
    all_data2[i]=le.fit_transform(all_data2[i])

In [ ]:
#Seperate train and test datasets with the same numbers of rows as original train and test datasets
train_2=all_data2[:len(train)]
test_2=all_data2[len(train):]

# Modelling

In [ ]:
#Create a random forest model
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=100, max_depth=5, max_features=0.7,  n_jobs=2)
#https://www.kaggle.com/competitions/mercedes-benz-greener-manufacturing/discussion/36128=< reference for parameter setting for the model
#Fitting the model and transforming the logarithm 
rf.fit(train_2,np.log(train["y"]))
result=rf.predict(test_2)
result

In [ ]:
#Can see how much independent variables (x) affects the dependent (outcome) variable 
rf.feature_importances_
pd.Series(rf.feature_importances_)
pd.Series(rf.feature_importances_,index=train_2.columns)
pd.Series(rf.feature_importances_,index=train_2.columns).sort_values(ascending=False)


# Submission

In [ ]:
sub=pd.read_csv("/kaggle/input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip")
sub

In [ ]:
sub["y"]=np.exp(result)
sub

In [ ]:
sub.to_csv("sub_2468.csv",index=0)

Private Score= 0.54858

Public Score= 0.55344

Approximately top 37% for private score category